# Lets figure out GIS

## Todo

- [ ] Get property boundaries loaded
- [ ] Get planning layers loaded
- [ ] Get heritage overlays loaded
- [ ] Delete earlier notebook

# Scrapping

Here I'm testing different configurations for scrapping.

## Different APIs to test

- [NSW Spatial][portal-nsw-spatial] `https://portal.spatial.nsw.gov.au`
  - **NSW Property feature layer**: this is of interest because it has NSW property id which can be used to link against the value general stuff.
  - **NSW Lot feature layer**: This is interesting as it has lot numbers.
- [Environment NSW][portal-nsw-environment]

[portal-nsw-environment]: https://mapprod3.environment.nsw.gov.au/arcgis/rest/services/Planning/EPI_Primary_Planning_Layers/MapServer
[portal-nsw-spatial]: https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer

## Request Times

- Querying NSW Address field
  - Filtering by `lastupdated` field
    - Jan 2023 to Jun 2024, 0h 3m 55s
    - Jan 2020 to Oct 2024, 0h 10m 39s 
    - Jan 2010 to Oct 2024, **4**h **16**m **29**s (`2410766` items)
      - Interestingly when I saw the finished result the last result was `April 2013`. At the time I thought that was the earlier it went back, but I think that must have been a timeframe with some of the most updates.
    - Feb 2013 to Apr 2013, **36**m **40**s (12 concurrent workers)
      - I checked this to see `April 2013` is really the last period
      - The first 2 months takes **2**m **27**s to run @ `25703` items
      - There must be a lot of stuff here
    - Jan 1990 to Feb 2013
      - `143600` items in it's at **35**m **24**s
      - got to `290100` items at **1**h **17**m **55**s and crashed on `2008-04-01`
    - Apr 2008 to May 2008
      - got to `249574` items at **1**h **1**m **4**s, still going
      - got to `422941` items at **1**h **53**m **42**s, then crashed

In [ ]:
from aiohttp import ClientSession
from datetime import datetime
import geopandas as gpd
from IPython.display import clear_output
import math
import matplotlib.pyplot as plt
import time

from lib.debug import NotebookTimer
from lib.gis.defaults import HOST_SEMAPHORE_CONFIG
from lib.gis.defaults import ENSW_DA_PROJECTION
from lib.gis.defaults import SNSW_LOT_PROJECTION
from lib.gis.defaults import SNSW_PROP_PROJECTION
from lib.gis.defaults import ENSW_ZONE_PROJECTION

from lib.gis.gis_streamer import GisStreamer, GisReaderError, ThrottledSession

try:
    timer_state = { 'count': 0, 'items': 0 }
    timer = NotebookTimer('#%(count)s: %(items)s items via', timer_state)
    
    async with ClientSession() as session:
        t_session = ThrottledSession.create(session, HOST_SEMAPHORE_CONFIG)
        reader = GisStreamer(t_session)
        # reader.queue(ENSW_ZONE_PROJECTION)
        # reader.queue(ENSW_DA_PROJECTION)
        reader.queue(SNSW_PROP_PROJECTION)
        # reader.queue(SNSW_LOT_PROJECTION)
        
        async for proj, task, page in reader.consume([]):
            clear_output(wait=True)
            timer.add({ 'items': len(page), 'count': 1 })
            timer.log()
            print(proj.schema.url)
            print(reader.progress(proj, task))
            if len(page) > 0:
                fig, ax = plt.subplots(1, 1, figsize=(2, 4))
                page.plot(ax=ax, column=proj.schema.debug_plot_column)
                plt.show()
                display(page.iloc[:1])
                
        print(f"finished loading GIS'")
except GisReaderError as e:
    print(e)